# Datorseenede projekt 
#### Projektet i kursen datorseende

In [13]:
%pip cache purge

Files removed: 884 (2914.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip uninstall torchvision -y


Found existing installation: torchvision 0.21.0+cu126
Uninstalling torchvision-0.21.0+cu126:
  Successfully uninstalled torchvision-0.21.0+cu126
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.


In [15]:

%pip install torchvision --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.1 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.1 MB 1.3 MB/s eta 0:00:05
   ----- ---------------------------------- 0.8/6.1 MB 1.3 MB/s eta 0:00:05
   ------ --------------------------------- 1.0/6.1 MB 1.3 MB/s eta 0:00:04
   -------- ------------------------------- 1.3/6.1 MB 1.3 MB/s eta 0:00:04
   ---------- ----------------------------- 1.6/6.1 MB 1.3 MB/s eta 0:00:04
   ----------- ---------------------------- 1.8/6.1 MB 1.3 MB/s eta 0:00:04
   ------------- -------------------------- 2.1/6.1 MB 1.3 MB/s eta 0:00:04
   --------------- ------------------------ 2.4/6.1 MB 1.3 MB/s eta 0:00:03
   ----------------- ---------------------- 2.6/6.1 MB 1.3 MB/s eta 0:00:03
   ------------------ --------------------- 2.9/6.1 MB 1.3 MB/s eta 0:00:03
   -------------------- -------------------

### Installationer
Installerar paket som behövs för exekveringen av koden.

In [16]:
%pip install split-folders
%pip install --upgrade ultralytics
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu126Note: you may need to restart the kernel to use updated packages.



### Spjälka data 
Spjälker data för att kunna användas i yolo modellen. Data spjälks till test, train och val.

In [7]:
import splitfolders

input_folder = "./input-data"
output_folder = "./dataset"

splitfolders.ratio(input_folder,output = output_folder, seed=42, ratio=(0.7,0.2,0.1),group_prefix=None)

Copying files: 5302 files [00:24, 212.72 files/s]


### Testa installation
för att kolla att allt är installerat korrekt och att versionerna matchar

In [19]:
import torch
import torchvision
print(torch.__version__)  # Ska vara 2.6.0+cu126
print(torchvision.__version__)  # Ska ha +cu126, inte +cpu
print(torch.cuda.is_available())  # Ska vara True


2.6.0+cu126
0.21.0+cu126
True


## Utför träningen 
Träna en yolov11 model med datan som vi har, med passande parametrar för egen dator att köra.

In [6]:
from ultralytics import YOLO

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11s.pt")

### Träna modelen på dataset

In [9]:
# Train model on custom dataset
results = model.train(
    data="config.yaml",
    project="./output",
    device='cuda:0',
    epochs=100,
    batch=-1,  
    imgsz=512,
    workers=12,
    amp=True, 
    name="yolo11_50",
    exist_ok=True,
    patience=10
)

New https://pypi.org/project/ultralytics/8.3.92 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.88  Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo11s.pt, data=config.yaml, epochs=100, time=None, patience=10, batch=-1, imgsz=512, save=True, save_period=-1, cache=False, device=cuda:0, workers=12, project=./output, name=yolo11_50, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_

train: Scanning E:\Programming\cv-25-project\dataset\train\labels... 1855 images, 16 backgrounds, 0 corrupt: 100%|██████████| 1855/1855 [00:04<00:00, 423.01it/s]


train: New cache created: E:\Programming\cv-25-project\dataset\train\labels.cache
AutoBatch: Computing optimal batch size for imgsz=512 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce GTX 1660) 6.00G total, 1.04G reserved, 0.10G allocated, 4.86G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     9428566       13.79         0.740         60.22         168.7        (1, 3, 512, 512)                    list
     9428566       27.58         5.499         46.09         613.4        (2, 3, 512, 512)                    list
     9428566       55.17         1.787         44.93         102.9        (4, 3, 512, 512)                    list
     9428566       110.3         3.087          89.9         176.1        (8, 3, 512, 512)                    list
     9428566       220.7         5.750         174.3         317.3       (16, 3, 512, 512)                    list
AutoBatch: Using batch-size 4 for CUDA:

train: Scanning E:\Programming\cv-25-project\dataset\train\labels.cache... 1855 images, 16 backgrounds, 0 corrupt: 100%|██████████| 1855/1855 [00:00<?, ?it/s]
val: Scanning E:\Programming\cv-25-project\dataset\val\labels... 530 images, 5 backgrounds, 0 corrupt: 100%|██████████| 530/530 [00:01<00:00, 464.13it/s]


val: New cache created: E:\Programming\cv-25-project\dataset\val\labels.cache
Plotting labels to output\yolo11_50\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 12 dataloader workers
Logging results to output\yolo11_50
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      5.53G     0.8821     0.5296     0.9568         11        512: 100%|██████████| 464/464 [01:10<00:00,  6.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:06<00:00, 10.24it/s]

                   all        530       1001      0.789      0.761       0.84       0.57



100 epochs completed in 2.315 hours.
Optimizer stripped from output\yolo11_50\weights\last.pt, 19.1MB
Optimizer stripped from output\yolo11_50\weights\best.pt, 19.1MB

Validating output\yolo11_50\weights\best.pt...
Ultralytics 8.3.88  Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660, 6144MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:06<00:00, 10.63it/s]


                   all        530       1001      0.792      0.761      0.841       0.57
                 skier        278        448      0.789      0.741      0.815      0.545
           snowboarder        351        553      0.794      0.782      0.867      0.596
Speed: 0.2ms preprocess, 5.7ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to output\yolo11_50


### Som följande kör vi validering på vår model

In [10]:
# Evaluate the model's performance on the validation set
results = model.val()

Ultralytics 8.3.88  Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660, 6144MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning E:\Programming\cv-25-project\dataset\val\labels.cache... 530 images, 5 backgrounds, 0 corrupt: 100%|██████████| 530/530 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 133/133 [00:06<00:00, 19.60it/s]


                   all        530       1001       0.79      0.761      0.841       0.57
                 skier        278        448      0.785      0.741      0.815      0.544
           snowboarder        351        553      0.794      0.782      0.867      0.595
Speed: 0.3ms preprocess, 6.8ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to output\yolo11_50
